In [1]:
from tensorflow.keras.optimizers import Adam
from keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger
from eval_utils import calculate_results, prepare_for_evaluation, generate_report, encode_sequences, encode_output
from dataloader import *
from  config_translator import *
from data_processor import preprocess_data
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Input
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

ModuleNotFoundError: No module named 'keras'

In [ ]:
data = DataLoader(config_mixed_coco14_coco14_glove)

In [ ]:
data=preprocess_data(data)

In [ ]:
# def data_generator(train_input, input_tokenizer, max_input_length, train_output, output_tokenizer, max_output_length,
#                    output_vocab_size, batch_size=64):
#     while 1:
#         count = 0
# 
#         while 1:
#             if count >= len(train_output):
#                 count = 0
# 
#             input_seq = list()
#             output_seq = list()
#             for i in range(count, min(len(train_input), count+batch_size)):
#                 input, output = train_input[i], train_output[i]
#                 input_seq.append(input)
#                 output_seq.append(output)
#             input_seq = encode_sequences(input_tokenizer, max_input_length, input_seq)
#             output_seq = encode_sequences(output_tokenizer, max_output_length, output_seq)
#             output_seq = encode_output(output_seq, output_vocab_size)
# 
#             count = count + batch_size
# 
#             input_seq = np.array(input_seq)
#             output_seq = np.array(output_seq)
#             output_seq = output_seq.reshape((output_seq.shape[0], output_seq.shape[1],output_vocab_size))
#             import tensorflow as tf
#             with tf.device('/cpu:0'):
#                 input_seq = tf.convert_to_tensor(input_seq, np.uint8)
#                 output_seq = tf.convert_to_tensor(output_seq, np.uint8)
#             yield [input_seq, output_seq]

def generate_batch(X,y , max_input_length, max_output_length, num_decoder_tokens,batch_size):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_input_length),dtype=np.float32)
            decoder_input_data = np.zeros((batch_size, max_output_length),dtype=np.float32)
            decoder_target_data = np.zeros((batch_size, max_output_length, num_decoder_tokens),dtype=np.float32)
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                encoder_input_data[i] = input_text # encoder input seq
                decoder_input_data[i] = target_text # decoder input seq
                for t, word in enumerate(target_text):
                    if word != 0:
                        decoder_target_data[i, t,word] = 1
            yield([encoder_input_data, decoder_input_data], decoder_target_data)


def encoder_model(num_words_output,LSTM_NODES):
    decoder_state_input_h = Input(shape=(LSTM_NODES,))
    decoder_state_input_c = Input(shape=(LSTM_NODES,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_inputs_single = Input(shape=(1,))
    decoder_embedding = Embedding(num_words_output, LSTM_NODES)
    decoder_inputs_single_x = decoder_embedding(decoder_inputs_single)
    decoder_lstm = LSTM(LSTM_NODES, return_sequences=True, return_state=True)
    decoder_outputs, h, c = decoder_lstm(decoder_inputs_single_x, initial_state=decoder_states_inputs)
    decoder_states = [h, c]
    decoder_dense = Dense(num_words_output, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs_single] + decoder_states_inputs,[decoder_outputs] + decoder_states)
    return encoder_model, decoder_model

def translate_sentence(input_seq, word2idx_inputs, word2idx_outputs, max_out_len, encoder_model, decoder_model):
    idx2word_input = {v:k for k, v in word2idx_inputs.items()}
    idx2word_target = {v:k for k, v in word2idx_outputs.items()}
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = word2idx_outputs['<sos>']
    eos = word2idx_outputs['<eos>']
    output_sentence = []

    for _ in range(max_out_len):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        idx = np.argmax(output_tokens[0, 0, :])

        if eos == idx:
            break

        word = ''

        if idx > 0:
            word = idx2word_target[idx]
            output_sentence.append(word)

        target_seq[0, 0] = idx
        states_value = [h, c]

    return ' '.join(output_sentence)


class ModelImpl:
    def __init__(self, data):
        self.data=data
        def define_model_glove(num_words, embedding_matrix, max_input_len,max_out_len, num_words_output, EMBEDDING_SIZE, LSTM_NODES=256):
            encoder_inputs_placeholder = Input(shape=(max_input_len,))
            embedding_layer = Embedding(num_words, EMBEDDING_SIZE, weights=[embedding_matrix], input_length=max_input_len)
            x = embedding_layer(encoder_inputs_placeholder)
            encoder = LSTM(LSTM_NODES, return_state=True)

            encoder_outputs, h, c = encoder(x)
            encoder_states = [h, c]
            decoder_inputs_placeholder = Input(shape=(max_out_len,))

            decoder_embedding = Embedding(num_words_output, LSTM_NODES)
            decoder_inputs_x = decoder_embedding(decoder_inputs_placeholder)

            decoder_lstm = LSTM(LSTM_NODES, return_sequences=True, return_state=True)
            decoder_outputs, _, _ = decoder_lstm(decoder_inputs_x, initial_state=encoder_states)
            decoder_dense = Dense(num_words_output, activation='softmax')
            decoder_outputs = decoder_dense(decoder_outputs)
            model = Model([encoder_inputs_placeholder, decoder_inputs_placeholder], decoder_outputs)
            return model

        self.model = define_model_glove(self.data.num_words, self.data.embedding_matrix_input,
                                        self.data.max_input_length, self.data.max_output_length,
                                        self.data.num_words_output, glove["embedings_dim"], 256  )
        self.model.compile(
            optimizer=self.optimizer(),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        self.model.summary()
        self.setup()

    def setup(self):
        # model.optimizer.lr = 0.001
        self.epochs = 100
        self.batch_size=200
        self.steps = len(self.data.output_sentences_list_with_start) // self.batch_size

    def optimizer(self):
        return Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    def train(self):
        model_weights_path="./" + self.data.configuration["data_name"] + self.data.configuration["model_save_dir"]
        if self.data.configuration["train_model"]:
            es = EarlyStopping(monitor='loss', min_delta=0.001, patience=3)
            filepath = model_weights_path + self.data.configuration["model_save_path"]
            checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True,
                                         mode='min', save_weights_only=False)
            callbacks_list = [checkpoint, es, CSVLogger("./" + self.data.configuration["data_name"] +'/logs.csv', separator=",", append=True),]
            if self.data.configuration["continue_training"]:
                self.model = load_model(filepath)
                print("New model loaded")

            train_generator = generate_batch(self.data.encoder_input_sequences, self.data.decoder_input_sequences,
                                             self.data.max_input_length, self.data.max_output_length,
                                             self.data.num_words_output, batch_size= self.batch_size)
            self.model.fit(train_generator,
                epochs=self.epochs,
                steps_per_epoch=self.steps,
                callbacks=[callbacks_list],
                verbose=1,
            )
            if self.data.configuration["save_model"]:
                writepath = model_weights_path+ "/"+'model' + '.h5'
                self.model.save(writepath)
                self.model.save_weights(model_weights_path
                                        + self.data.configuration["model_save_path"])

    def evaluate(self):
        model_weights_path="./" + self.data.configuration["data_name"] + self.data.configuration["model_save_dir"]
        print("Model loaded from file")
        self.model.load_weights(model_weights_path + self.data.configuration["model_save_path"])
        expected, results = prepare_for_evaluation(self.data, self.model)
        out = calculate_results(expected, results, self.data.configuration)
        print(out)
model=ModelImpl(data)

In [ ]:
model.train()

In [ ]:
model.evaluate()

In [ ]:
generate_report(general["results_directory"])